# LightGBM model with Airline dataset. Experiment 01

LightGBM regressor with logloss + Airline dataset

In [3]:
import os,sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from lightgbm.sklearn import LGBMRegressor
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import (confusion_matrix, accuracy_score, roc_auc_score, f1_score, log_loss, precision_score,
                             recall_score, mean_squared_error, mean_absolute_error, r2_score)
from scipy.stats import gmean

from libs.timer import Timer
from libs.loaders import load_fraud, load_iot, load_airline
from libs.conversion import convert_cols_categorical_to_numeric, convert_related_cols_categorical_to_numeric
from libs.conversion import _get_nominal_integer_dict, _convert_to_integer

print("System version: {}".format(sys.version))

System version: 3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]


### Load data

In [6]:
%%time
df_plane = load_airline()
print(df_plane.shape)


(115069017, 14)
CPU times: user 1min 16s, sys: 1min 17s, total: 2min 33s
Wall time: 4min 47s


In [7]:
df_plane.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,AA,190,247,SFO,ORD,1846,0,27
1,1987,10,1,4,5,114,EA,57,74,LAX,SFO,337,0,5
2,1987,10,1,4,5,35,HP,351,167,ICT,LAS,987,0,17
3,1987,10,1,4,5,40,DL,251,35,MCO,PBI,142,0,-2
4,1987,10,1,4,8,517,UA,500,208,LAS,ORD,1515,0,17


In [8]:
%%time
df_plane_numeric = convert_related_cols_categorical_to_numeric(df_plane, col_list=['Origin','Dest'])


CPU times: user 1min 13s, sys: 1min, total: 2min 14s
Wall time: 2min 17s


In [9]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,AA,190,247,0,33,1846,0,27
1,1987,10,1,4,5,114,EA,57,74,1,0,337,0,5
2,1987,10,1,4,5,35,HP,351,167,2,4,987,0,17
3,1987,10,1,4,5,40,DL,251,35,3,41,142,0,-2
4,1987,10,1,4,8,517,UA,500,208,4,33,1515,0,17


In [10]:
%%time
df_plane_numeric = convert_cols_categorical_to_numeric(df_plane_numeric, col_list='UniqueCarrier')


CPU times: user 38.2 s, sys: 37.3 s, total: 1min 15s
Wall time: 1min 17s


In [11]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,0,190,247,0,33,1846,0,27
1,1987,10,1,4,5,114,1,57,74,1,0,337,0,5
2,1987,10,1,4,5,35,2,351,167,2,4,987,0,17
3,1987,10,1,4,5,40,3,251,35,3,41,142,0,-2
4,1987,10,1,4,8,517,4,500,208,4,33,1515,0,17


In [12]:
%%time
filter_func = lambda x: 1 if x > 0 else 0
df_plane_numeric['ArrDelayBinary'] = df_plane_numeric['ArrDelay'].map(filter_func)

CPU times: user 23.3 s, sys: 36.2 s, total: 59.5 s
Wall time: 1min


In [13]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay,ArrDelayBinary
0,1987,10,1,4,1,556,0,190,247,0,33,1846,0,27,1
1,1987,10,1,4,5,114,1,57,74,1,0,337,0,5,1
2,1987,10,1,4,5,35,2,351,167,2,4,987,0,17,1
3,1987,10,1,4,5,40,3,251,35,3,41,142,0,-2,0
4,1987,10,1,4,8,517,4,500,208,4,33,1515,0,17,1


In [14]:
%%time
#gets a random 80% of the entire set
X_train = df_plane_numeric.sample(frac=0.8, random_state=1)
print(X_train.shape)
#gets the left out portion of the dataset
X_test = df_plane_numeric.loc[~df_plane_numeric.index.isin(X_train.index)]
print(X_test.shape)

(92055214, 15)
(23013803, 15)
CPU times: user 1min 34s, sys: 2min 59s, total: 4min 34s
Wall time: 4min 40s


### Training

In [15]:
# Variables
KFOLD_SPLITS = 5

In [ ]:
def loglikelood(y_true, y_pred):
    eval_result = log_loss(y_true, y_pred)
    eval_name = 'log_loss'
    is_bigger_better = False
    return eval_name, eval_result, is_bigger_better

In [16]:
def train_lightgbm(df, verbose=True):
    if verbose: print("Training data of size {}".format(df.shape))
    x = df[df.columns.difference(['ArrDelay', 'ArrDelayBinary'])].as_matrix()
    y = df['ArrDelayBinary'].as_matrix()
    
    n_splits = 5
    skf = KFold(n_splits=n_splits, random_state=77, shuffle=True)
    clfs = []
    
    if verbose: print("Computing boosted tree using {} kfold cross validation".format(n_splits))
    for train_index, test_index in skf.split(x, y):
        trn_x, val_x = x[train_index,:], x[test_index,:]
        trn_y, val_y = y[train_index], y[test_index]

        clf = LGBMRegressor(num_leaves=255,
                            n_estimators=100,
                            min_child_weight=30,
                            learning_rate=0.1,
                            nthread=20,
                            subsample=0.80,
                            colsample_bytree=0.80,
                            seed=77)
        clf.fit(trn_x, trn_y, eval_set=[(val_x, val_y)], verbose=verbose, eval_metric=loglikelood)
        clfs.append(clf)
        
    return clfs

In [17]:
#X_train_small = X_train.sample(frac=0.001, random_state=1)
#print(X_train_small.shape)

In [18]:
%%time
clfs = train_lightgbm(X_train)

Training data of size (92055214, 15)
Computing LightGBM boosted tree using 5 kfold cross validation
[1]	valid_0's l2: 0.49923
Train until valid scores didn't improve in 100 rounds.
[2]	valid_0's l2: 0.49881
[3]	valid_0's l2: 0.498403
[4]	valid_0's l2: 0.497992
[5]	valid_0's l2: 0.497573
[6]	valid_0's l2: 0.497183
[7]	valid_0's l2: 0.496882
[8]	valid_0's l2: 0.496778
[9]	valid_0's l2: 0.496451
[10]	valid_0's l2: 0.496161
[11]	valid_0's l2: 0.495845
[12]	valid_0's l2: 0.495739
[13]	valid_0's l2: 0.495642
[14]	valid_0's l2: 0.495281
[15]	valid_0's l2: 0.495196
[16]	valid_0's l2: 0.495047
[17]	valid_0's l2: 0.494749
[18]	valid_0's l2: 0.494652
[19]	valid_0's l2: 0.494536
[20]	valid_0's l2: 0.4944
[21]	valid_0's l2: 0.49426
[22]	valid_0's l2: 0.494145
[23]	valid_0's l2: 0.493781
[24]	valid_0's l2: 0.4935
[25]	valid_0's l2: 0.493227
[26]	valid_0's l2: 0.492885
[27]	valid_0's l2: 0.492754
[28]	valid_0's l2: 0.492451
[29]	valid_0's l2: 0.492158
[30]	valid_0's l2: 0.492046
[31]	valid_0's l2: 0.

### Evaluation


In [19]:
def compute_prediction(clfs, df, verbose=True):    
    x = df[df.columns.difference(['ArrDelay', 'ArrDelayBinary'])].as_matrix()            
    preds = []
    with Timer() as t:
        for clf in clfs:
            preds.append(np.clip(clf.predict(x), 0.0001, 0.9999))
        pred = gmean(np.array(preds), axis=0)
    if verbose: print("Prediction took %.03f sec.\n" % t.interval)        
    df['prediction'] = pred
    return df

In [20]:
#X_test_small = X_test.sample(frac=0.001, random_state=1)
#print(X_test_small.shape)

In [21]:
%%time
X_test_pred = compute_prediction(clfs, X_test)
X_test_pred.head(20)

Prediction took 24692.342 sec.

CPU times: user 4h 38min 14s, sys: 2h 13min 18s, total: 6h 51min 33s
Wall time: 21min 25s


/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### Metrics

In [22]:
#https://github.com/miguelgfierro/codebase/blob/master/python/machine_learning/metrics.py
def classification_metrics_binary(y_true, y_pred):
    m_acc = accuracy_score(y_true, y_pred)
    m_f1 = f1_score(y_true, y_pred)
    m_precision = precision_score(y_true, y_pred)
    m_recall = recall_score(y_true, y_pred)
    m_conf = confusion_matrix(y_true, y_pred)
    report = {'Accuracy':m_acc, 'Precision':m_precision, 'Recall':m_recall, 'F1':m_f1, 'Confusion Matrix':m_conf}
    return report

In [23]:
#https://github.com/miguelgfierro/codebase/blob/master/python/machine_learning/metrics.py
def classification_metrics_binary_prob(y_true, y_prob):
    m_auc = roc_auc_score(y_true, y_prob)
    m_logloss = log_loss(y_true, y_prob)
    report = {'AUC':m_auc, 'Log loss':m_logloss}
    return report

In [24]:
THRES = 0.5
threshold_func = lambda x: 0 if x<=THRES else 1


In [25]:
y_true = X_test_pred['ArrDelayBinary'].as_matrix()
y_prob = X_test_pred['prediction'].as_matrix()
y_pred = X_test_pred['prediction'].map(threshold_func).as_matrix()

In [26]:
report1 = classification_metrics_binary(y_true, y_pred)
report1

{'Accuracy': 0.67855160661625546,
 'Confusion Matrix': array([[9492952, 2600347],
        [4797403, 6123101]]),
 'F1': 0.6234082632659661,
 'Precision': 0.70191293626098306,
 'Recall': 0.56069765644516045}

In [27]:
report2 = classification_metrics_binary_prob(y_true, y_prob)
report2

{'AUC': 0.7405583946658062, 'Log loss': 0.61727332902949317}